In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

# Load the dataset
data = pd.read_csv("your_dataset.csv")  # Replace "your_dataset.csv" with the path to your dataset file

# Convert categorical variables into numerical using one-hot encoding
data = pd.get_dummies(data, columns=['Gender', 'Category', 'Season'])

# Split features and target variable
X = data.drop(columns=['Item Purchased','Location'])
y = data['Item Purchased']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
joblib.dump(scaler, 'scaler(nolocation).pkl')

X_test_scaled = scaler.transform(X_test)

# Train the KNN model
k = 5  # Number of neighbors
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train_scaled, y_train)


# Save the model to a file
joblib.dump(knn_model, 'knn_personalzied_recommendation(no location).pkl')


# Make predictions on the testing data
y_pred = knn_model.predict(X_test_scaled)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Function to make recommendations based on input features
def make_recommendations(age, gender, category,  season, top_n=10):
    # Convert input features into numerical values using one-hot encoding
    input_data = pd.DataFrame({'Age': [age], 'Gender_' + gender: [1], 'Category_' + category: [1],
                                'Season_' + season: [1]})
    input_data = input_data.reindex(columns=X.columns, fill_value=0)
    
    # Scale the input features
    input_data_scaled = scaler.transform(input_data)
    
    # Predict the probabilities for all classes
    probabilities = knn_model.predict_proba(input_data_scaled)
    
    top_10_indices = probabilities.argsort()[0][-10:][::-1]

    top_10_items = knn_model.classes_[top_10_indices]

    return top_10_items

from datetime import datetime
def get_current_season():
    month = datetime.now().month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Example usage:
age = 60
gender = 'Male'
category = 'Clothing'
season = get_current_season()
recommendations = make_recommendations(age, gender, category, season, top_n=10)
print("Top 10 recommended items:")
print(recommendations)




Accuracy: 0.16153846153846155
Top 10 recommended items:
['T-shirt' 'Skirt' 'Shorts' 'Dress' 'Jacket' 'Belt' 'Blouse' 'Boots'
 'Coat' 'Gloves']
